In [4]:
import numpy as np
import pandas as pd
import re

from IPython.display import clear_output

#!pip install "gensim"
from gensim.models import Word2Vec   # Word2Vec model

In [12]:
df = pd.read_csv("/home/x-mbemus/Desktop/text_train_data.csv")   # Open training dataset.
df.head()

,Unnamed: 0,id,title,author,pub_date,genres,summary,link,is_novel,sum_len,file_name
0,2,6921,Carmilla,Sheridan Le Fanu,1872,fiction,The story is presented by Le Fanu as part of ...,https://www.gutenberg.org/files/10007/10007-0.txt,False,934,10007_10007-0.txt
1,3,7923,Dracula,Bram Stoker,1897,horror,"The novel is told in epistolary format, as a ...",https://www.gutenberg.org/files/345/345-0.txt,True,986,345_345-0.txt
2,4,7923,Dracula,Bram Stoker,1897,horror,"The novel is told in epistolary format, as a ...",https://www.gutenberg.org/files/45839/45839-0.txt,True,986,45839_45839-0.txt
3,5,8237,Don Quixote,Miguel de Cervantes,1605,novel,"The First Sally Alonso Quijano, the protagoni...",https://www.gutenberg.org/files/996/996-0.txt,True,1592,996_996-0.txt
4,9,13535,Heart of Darkness,Joseph Conrad,1899,novel,'Heart of Darkness' opens in first person nar...,https://www.gutenberg.org/files/219/219-0.txt,True,3127,219_219-0.txt


In [ ]:
sents = []   # List to hold sentences.
for i in df.loc[:]["file_name"]:   # For each file.
    new_sent = []   # Create new sentence.
    try:
        # Open file.
        f = open("/home/x-mbemus/Desktop/text_files/" + i, "r", errors="ignore")
        text = ' '.join(f.readlines())   # Read whole file.
        # Text preprocessing.
        text = re.sub(r"\\n|\\t", " ", text)   # Remove any new-lines and tabs.
        text = re.sub(r"([^\w\s'])", r" \1 ", text)   # Put spaces around most punctuation.
        text = re.sub(r"\s+", " ", text)   # Remove extra spaces.
        words = text.split(" ")   # Separate tokens by space.
        for j in words:   # For each token.
            new_sent.append(j)   # Add token to sentence.
            if j in [".", "!", "?"]:   # If token is punctuation.
                sents.append(new_sent)   # Add sentence to sentence list.
                new_sent = []   # Create new sentence.
        if len(new_sent) > 0:   # If current sentence not empty.
            sents.append(new_sent)   # Add to sentence list.
        f.close()    # Close file.
    except:
        pass

In [43]:
sents_2 = []    # Getting summary sentences.
new_sent = []   # Individual sentence holder.
for i in df.loc[:]["summary"]:   # Iterate through sentences.
    # Same preprocessing as before.
    text = re.sub(r"\\n|\\t", " ", i)
    text = re.sub(r"\\'", " ", text)
    text = re.sub(r"([^\w\s'])", r" \1 ", text)
    text = re.sub(r"\s+", " ", text)
    words = text.split(" ")    # Split tokens by space.
    for j in words:    # Iterate through tokens.
        new_sent.append(j)   # Add word to current sentence.
        if j in [".", "!", "?"]:   # If punctuation.
            sents_2.append(new_sent)   # Add sentence to sentence list.
            new_sent = []   # Create new sentence.
    if len(new_sent) > 0:   # If new sentence not empty.
        sents_2.append(new_sent)   # Add incomplete sentence to list.

In [44]:
all_sents = sents + sents_2   # Combine all sentences.
print(len(all_sents))   # Check count of sentences.

2922600


In [45]:
# Train 50-vec from sentences.
model = Word2Vec(sentences=all_sents, vector_size=50, window=5, min_count=1, workers=4)

In [48]:
model.wv['Dracula']   # Test observation.

array([-0.38713482,  0.35845155, -0.4892324 ,  0.10323673,  0.21779941,
        0.18955876,  0.6996074 ,  0.22049618, -0.26312035,  0.28026226,
        0.41193014, -0.18090388, -0.24702743,  0.483984  ,  0.01322602,
        0.23626558,  0.3272087 , -0.00264295, -0.24421367, -0.5000111 ,
        0.2437251 ,  0.67580444, -0.16511494,  0.21222222,  0.33947915,
        0.04529685, -0.01860966,  0.05741313, -0.47610345,  0.07334362,
        0.04463233, -0.32907084, -0.11431769, -0.44045985, -0.22120006,
        0.1330758 ,  0.21142244,  0.14917983,  0.34179702,  0.4040718 ,
        0.17557351, -0.05840934, -0.0798966 ,  0.17631772,  0.3998845 ,
       -0.08932945,  0.17433514,  0.02371125, -0.05882114,  0.4896915 ],
      dtype=float32)

In [47]:
model.save("train-text-50.vecs")   # Save model.